# DBSCAN

## Implementation

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
class DBSCAN(object):
    UNCLASSIFIED = -2
    NOISE = -1
    
    def __init__(self, eps=0.5, min_pts=5):
        self.eps = eps
        self.min_pts = min_pts

    def fit(self, X):
        labels = np.full([X.shape[0]], DBSCAN.UNCLASSIFIED)
        return self._fit(X, labels)

    def _fit(self, X, labels):
        cluster_id = DBSCAN._next_id(DBSCAN.NOISE)
        for point_ind in range(X.shape[0]):
            if labels[point_ind] == DBSCAN.UNCLASSIFIED:
                if self._expand_cluster(X, labels, point_ind, cluster_id):
                    cluster_id = DBSCAN._next_id(cluster_id)
        return labels

    def _expand_cluster(self, X, labels, point_ind, cluster_id):
        region_inds = self._region_query(X, point_ind)
        if len(region_inds) < self.min_pts:
            labels[point_ind] = DBSCAN.NOISE
            return False

        # label these points to cluseter_id
        labels[region_inds] = cluster_id
        labels[point_ind] = cluster_id

        queue_inds = deque(region_inds)
        while len(queue_inds):
            current_point_ind = queue_inds.popleft()

            result = self._region_query(X, current_point_ind)
            if len(result) > self.min_pts:
                is_noise = labels[result] == DBSCAN.NOISE
                is_unclassified = labels[result] == DBSCAN.UNCLASSIFIED
                
                # we add only unclussified points
                queue_inds.extend(result[is_unclassified])
                # label these points to cluseter_id
                labels[result[np.logical_or(is_noise, is_unclassified)]] = cluster_id
        return True

    def _region_query(self, X, point_ind):
        d = np.sqrt(np.sum((X - X[point_ind]) ** 2, axis=1))
        mask = d < self.eps
        mask[point_ind] = False  # exclude this point
        return np.where(mask)[0]

    @staticmethod
    def _next_id(cluster_id):
        return cluster_id + 1
